### Examples Of Pyspark ML

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [3]:
spark = SparkSession.builder.appName('Missing').getOrCreate()

25/01/01 17:24:47 WARN Utils: Your hostname, rakesh resolves to a loopback address: 127.0.1.1; using 192.168.1.103 instead (on interface wlp4s0)
25/01/01 17:24:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/01 17:24:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/01 17:24:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
## Read The dataset
training = spark.read.csv('test1.csv',header=True,inferSchema=True)
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [5]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [6]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [16]:
training.columns[1:-1]

['age', 'Experience']

In [18]:
## [Age,Experience]----> new feature--->independent feature
# featureassembler = VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")
featureassembler = VectorAssembler(inputCols=training.columns[1:-1], outputCol="Independent Features")
output = featureassembler.transform(training)
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [8]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [9]:
finalized_data = output.select(["Independent Features","Salary"])
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [10]:
##train test split
train_data,test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

25/01/01 17:24:56 WARN Instrumentation: [0122b73f] regParam is zero, which might cause numerical instability and overfitting.
25/01/01 17:24:57 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [11]:
### Coefficients
regressor.coefficients

DenseVector([-357.1429, 1785.7143])

In [12]:
### Intercepts
regressor.intercept

23214.285714287067

In [13]:
### Prediction
pred_results = regressor.evaluate(test_data)

In [14]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000|17500.000000000196|
|          [23.0,2.0]| 18000|  18571.4285714287|
|          [30.0,8.0]| 25000|26785.714285714275|
+--------------------+------+------------------+



In [15]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(1619.0476190477239, 3255102.040816691)

25/01/01 17:25:03 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
